<a href="https://colab.research.google.com/github/bubinsabu/mnist_digits_classification/blob/main/CIFAR_10_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Load and Normalize

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision.transforms import transforms
import sys

# Define Data transformation for Normalisation

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.5,0.5,0.5], std = [0.5,0.5,0.5])
])

In [3]:
# load and training CIFAR-10 datasets
trainset = torchvision.datasets.CIFAR10(root = "./data", train = True, download = True, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 64, shuffle = False)

# load and testing CIFAR-10 datasets
testset = torchvision.datasets.CIFAR10(root = "./data", train = False , download = True, transform = transform)
testloader = torch.utils.data.DataLoader(testset, batch_size = 64, shuffle = False)

classes = ("airplane", "automobile", "bird", "car","deer", "dog", "frog", "horse", "ship", "truck")

100%|██████████| 170498071/170498071 [00:02<00:00, 80143397.85it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
print("Train_Data_Shape: ",trainset.data.shape)
print("Test_Data_Shape: ",testset.data.shape)

Train_Data_Shape:  (50000, 32, 32, 3)
Test_Data_Shape:  (10000, 32, 32, 3)


# Define CNN

In [5]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16 * 5 * 5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84,10)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16 * 5 * 5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

net = Net()

# Define Loss Function and Optimizer

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.001, momentum = 0.9)

# Train the Model

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)

for epoch in range(10):
  running_loss = 0.0
  for i, data in enumerate(trainloader, 0):
    inputs, labels = data[0].to(device), data[1].to(device)

    optimizer.zero_grad()

    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if i % 2000 == 1999:
      print(f"[Epoch {epoch + 1},Iteration {i + 1}]loss:{running_loss / 2000:.3f}")
      running_loss = 0.0
      sys.stdout.flush()

In [13]:
print(f"[Epoch {epoch + 1},Iteration {i + 1}]loss:{running_loss / 2000:.3f}")

[Epoch 10,Iteration 782]loss:0.303
